## Model Training

### Environment setting

In [10]:
import pickle
import os
import itertools
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from utilsML import run_experiments, expand_parameters

### Read processed data

In [11]:
cache_file = "procData.pkl"
cache_dir = "cache"
try:
    with open(os.path.join(cache_dir, cache_file), "rb") as f:
        cache_data = pickle.load(f)
    print("Read preprocessed data from cache file:", cache_file)
except:
    pass

Read preprocessed data from cache file: procData.pkl


In [12]:
df = cache_data['proc_data']
df.head(5).T

,0,1,2,3,4
PassengerId,0001_01,0002_01,0003_01,0003_02,0004_01
HomePlanet,Europa,Earth,Europa,Europa,Earth
CryoSleep,False,False,False,False,False
Cabin,B/0/P,F/0/S,A/0/S,A/0/S,F/1/S
Destination,TRAPPIST-1e,TRAPPIST-1e,TRAPPIST-1e,TRAPPIST-1e,TRAPPIST-1e
Age,0.711945,-0.334037,2.036857,0.293552,-0.891895
VIP,False,False,True,False,False
RoomService,-0.333105,-0.168073,-0.268001,-0.333105,0.125652
FoodCourt,-0.281027,-0.275387,1.959998,0.52301,-0.237159
ShoppingMall,-0.283579,-0.241771,-0.283579,0.336851,-0.031059


In [13]:
X_train = df.drop('Transported', axis=1)
y_train = df['Transported']

### Basic initial models

Initially, we will evaluate two basic models, each relying on a single predictor variable strongly linked to the target: 'CryoSleep' and 'log10TotalExpenses1'. Predictions are made directly from these inputs and metrics for the classification are calculated.

#### 'CryoSleep' feature

This 'model' achieves an accuracy of 0.72

In [14]:
le = LabelEncoder()
y_true = le.fit_transform(df['Transported'])  
y_pred = le.fit_transform(df['CryoSleep'])   

print(pd.crosstab(df['Transported'], df['CryoSleep']))
print(classification_report(y_true, y_pred))

CryoSleep    False  True 
Transported              
False         3761    554
True          1895   2483
              precision    recall  f1-score   support

           0       0.66      0.87      0.75      4315
           1       0.82      0.57      0.67      4378

    accuracy                           0.72      8693
   macro avg       0.74      0.72      0.71      8693
weighted avg       0.74      0.72      0.71      8693



#### 'log10_TotalExpenses1' feature

This 'model' uses a feature calculated from 'TotalExpenses' feature. Samples are classified into two groups based on whether the log10 of the TotalExpenses variable is greater than or less than 1. This 'model' achieves an accuracy of 0.74.

In [15]:
le = LabelEncoder()
y_true = le.fit_transform(df['Transported'])  
y_pred = le.fit_transform(df['log10_TotalExpenses1'])   

print(pd.crosstab(df['Transported'], df['log10_TotalExpenses1']))
print(classification_report(y_true, y_pred))

log10_TotalExpenses1   0.0   1.0
Transported                     
False                 3530   785
True                  1505  2873
              precision    recall  f1-score   support

           0       0.70      0.82      0.76      4315
           1       0.79      0.66      0.72      4378

    accuracy                           0.74      8693
   macro avg       0.74      0.74      0.74      8693
weighted avg       0.74      0.74      0.73      8693



### Decision trees models

Feature combinations to test are chosen according to the EDA findings.

In [16]:
onehot_homeplanet = (list(filter(lambda x: any(map(lambda y: y in x,['HomePlanet_'])), df.columns)))
main_homeplanet = list(filter(lambda x: any(map(lambda y: y not in x, ['Mars'])),onehot_homeplanet))
onehot_destination = (list(filter(lambda x: any(map(lambda y: y in x,['Destination_'])), df.columns)))
main_destination = list(filter(lambda x: any(map(lambda y: y not in x, ['PSO'])),onehot_destination))
expenses_features = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
log10_expenses_features = ['log10_RoomService', 'log10_FoodCourt', 'log10_ShoppingMall', 'log10_Spa', 'log10_VRDeck']
log10_expenses_features1 = ['log10_RoomService1', 'log10_FoodCourt1', 'log10_ShoppingMall1', 'log10_Spa1', 'log10_VRDeck1']

cryosleepAge12 = ['CryoSleep_True','Age12','Deck_B','Side_S'] + main_homeplanet + main_destination
log10TE = ['log10_TotalExpenses','Deck_B','Side_S'] + main_homeplanet + main_destination
log10TE1 = ['log10_TotalExpenses1','Deck_B','Side_S'] + main_homeplanet + main_destination
expenses = expenses_features + ['Deck_B','Side_S'] + main_homeplanet + main_destination
log10Expenses = log10_expenses_features + ['Deck_B','Side_S'] + main_homeplanet + main_destination
log10Expenses1 = log10_expenses_features1 + ['Deck_B','Side_S'] + main_homeplanet + main_destination



The 'max_depth' hyperparameter is tested with various values, using all the feature combinations described above.

In [17]:
# combinations of model to check (algorithm + parameters)
dataproc = {'cryosleepAge12': cryosleepAge12, 'log10TE': log10TE, 'log10TE1': log10TE1, 
            'expenses': expenses, 'log10Expenses': log10Expenses, 'log10Expenses1': log10Expenses1} 
experiments = {
    'model': ['DecisionTree'],
    'algorithms': [DecisionTreeClassifier(criterion='gini')],
    'parameters': [{'max_depth': np.arange(2, 15)}]
}
results = run_experiments(dataproc, experiments, X_train, y_train)

The highest performance is achieved with the 'expenses', 'log10Expenses', and 'log10Expenses1' datasets.

In [18]:
max_indices = pd.DataFrame(results).drop(['features'],axis=1).groupby('feature_name')['test_score'].idxmax()
df_max = pd.DataFrame(results).drop(['features'],axis=1).loc[max_indices]
expand_parameters(df_max)

,feature_name,model,train_score,test_score,fit_time,max_depth
5,cryosleepAge12,DecisionTree,0.737145,0.734502,0.004797,7
43,expenses,DecisionTree,0.797193,0.776030,0.015629,6
56,log10Expenses,DecisionTree,0.797193,0.775915,0.015902,6
69,log10Expenses1,DecisionTree,0.759951,0.754288,0.006161,6
13,log10TE,DecisionTree,0.737145,0.737148,0.004703,2
26,log10TE1,DecisionTree,0.736570,0.736573,0.003835,2


It is reduced to the allowed range of values for 'max_depth' and tested with the hyperparameters 'min_samples_leaf', 'min_samples_split', and 'splitter'.

In [63]:
# combinations of model to check (algorithm + parameters)
dataproc = {'cryosleepAge12': cryosleepAge12, 'log10TE': log10TE, 'log10TE1': log10TE1, 
            'expenses': expenses, 'log10Expenses': log10Expenses, 'log10Expenses1': log10Expenses1} 
experiments = {
    'model': ['DecisionTree'],
    'algorithms': [DecisionTreeClassifier(criterion='gini')],
    'parameters': [{'max_depth': np.arange(2, 8),
                    'min_samples_leaf': np.arange(2,10),
                    'min_samples_split': np.arange(2,10),
                    'splitter': ['best','random']}]
}
results = run_experiments(dataproc, experiments, X_train, y_train)

When optimization of more hyperparameters is allowed, better score values are achieved. The best values still appear for the feature sets 'expenses', 'log10Expenses', and 'log10Expenses1'.

In [64]:
df = pd.DataFrame(results).drop(['features'],axis=1)
df_max3 = df.sort_values(['feature_name', 'test_score'], ascending=[True, False]).groupby('feature_name').head(3)

expand_parameters(df_max3)

,feature_name,model,train_score,test_score,fit_time,max_depth,min_samples_leaf,min_samples_split,splitter
576,cryosleepAge12,DecisionTree,0.736886,0.735307,0.005291,6,6,2,best
577,cryosleepAge12,DecisionTree,0.736886,0.735307,0.005137,6,6,2,random
578,cryosleepAge12,DecisionTree,0.736886,0.735307,0.005084,6,6,3,best
2848,expenses,DecisionTree,0.796474,0.776950,0.015591,6,4,2,best
2850,expenses,DecisionTree,0.796474,0.776950,0.015459,6,4,3,best
2852,expenses,DecisionTree,0.796474,0.776950,0.015357,6,4,4,best
3605,log10Expenses,DecisionTree,0.783188,0.779939,0.006516,6,3,4,random
3785,log10Expenses,DecisionTree,0.788364,0.779250,0.007177,7,6,6,random
3717,log10Expenses,DecisionTree,0.790550,0.778215,0.007169,7,2,4,random
4464,log10Expenses1,DecisionTree,0.759117,0.754748,0.006174,6,9,2,best


In [ ]:
# TODO Why 'log10TE1' 2, 2, 2???

Peak scores per dataset:

- 'expense': max_depth=6, min_samples_leaf=4 and any min_samples_split

- 'log10Expenses1': max_depth=6, min_samples_leaf=9 and any min_samples_split

- 'log10Expenses': max_depth=7, min_samples_leaf=7, min_samples_split=3

In [34]:
df = pd.DataFrame(results).drop(['features'], axis=1)
df_sel = df[df['feature_name'].isin(['expenses','log10Expenses','log10Expenses1'])]
df_max3 = df_sel.sort_values(['feature_name', 'test_score'], ascending=[True, False]).groupby('feature_name').head(3)

expand_parameters(df_max3)

,feature_name,model,train_score,test_score,fit_time,max_depth,min_samples_leaf,min_samples_split,splitter
2848,expenses,DecisionTree,0.796474,0.776950,0.015259,6,4,2,best
2850,expenses,DecisionTree,0.796474,0.776950,0.015286,6,4,3,best
2852,expenses,DecisionTree,0.796474,0.776950,0.016735,6,4,4,best
3779,log10Expenses,DecisionTree,0.788853,0.781551,0.007156,7,6,3,random
3801,log10Expenses,DecisionTree,0.787070,0.780630,0.007850,7,7,6,random
3703,log10Expenses,DecisionTree,0.782095,0.778099,0.006432,6,9,5,random
4464,log10Expenses1,DecisionTree,0.759117,0.754748,0.005925,6,9,2,best
4465,log10Expenses1,DecisionTree,0.759117,0.754748,0.006153,6,9,2,random
4466,log10Expenses1,DecisionTree,0.759117,0.754748,0.006017,6,9,3,best


In [40]:
df = pd.DataFrame(results).drop(['features'], axis=1)
df_sel = df[df['feature_name'].isin(['expenses'])]
df_max10 = df_sel.sort_values(['test_score'], ascending=[False]).head(10)

expand_parameters(df_max10)

,feature_name,model,train_score,test_score,fit_time,max_depth,min_samples_leaf,min_samples_split,splitter
2850,expenses,DecisionTree,0.796474,0.77695,0.015286,6,4,3,best
2852,expenses,DecisionTree,0.796474,0.77695,0.016735,6,4,4,best
2854,expenses,DecisionTree,0.796474,0.77695,0.016244,6,4,5,best
2856,expenses,DecisionTree,0.796474,0.77695,0.016251,6,4,6,best
2858,expenses,DecisionTree,0.796474,0.77695,0.016141,6,4,7,best
2860,expenses,DecisionTree,0.796474,0.77695,0.015325,6,4,8,best
2862,expenses,DecisionTree,0.796474,0.77695,0.015268,6,4,9,best
2848,expenses,DecisionTree,0.796474,0.77695,0.015259,6,4,2,best
2834,expenses,DecisionTree,0.796590,0.77672,0.015328,6,3,3,best
2832,expenses,DecisionTree,0.796590,0.77672,0.015479,6,3,2,best


In the previous calculation, some of the best models reached the maximum allowed value for 'min_samples_leaf'. We now test with an extended range for this parameter while reducing the ranges of other hyperparameters to avoid excessive computation time.

In [42]:
# combinations of model to check (algorithm + parameters)
dataproc = {'log10Expenses': log10Expenses, 'log10Expenses1': log10Expenses1} 
experiments = {
    'model': ['DecisionTree'],
    'algorithms': [DecisionTreeClassifier(criterion='gini')],
    'parameters': [{'max_depth': np.arange(5, 8),
                    'min_samples_leaf': np.arange(5,15),
                    'min_samples_split': np.arange(2,10),
                    'splitter': ['best','random']}]
}
results = run_experiments(dataproc, experiments, X_train, y_train)

Similar results has been achieved for new ranges.

In [45]:
df = pd.DataFrame(results).drop(['features'],axis=1)
df_max3 = df.sort_values(['feature_name', 'test_score'], ascending=[True, False]).groupby('feature_name').head(3)
expand_parameters(df_max3)

,feature_name,model,train_score,test_score,fit_time,max_depth,min_samples_leaf,min_samples_split,splitter
455,log10Expenses,DecisionTree,0.785891,0.779479,0.006959,7,13,5,random
343,log10Expenses,DecisionTree,0.787387,0.778905,0.007002,7,6,5,random
403,log10Expenses,DecisionTree,0.785460,0.778674,0.006960,7,10,3,random
704,log10Expenses1,DecisionTree,0.759117,0.754748,0.006051,6,9,2,best
705,log10Expenses1,DecisionTree,0.759117,0.754748,0.006462,6,9,2,random
706,log10Expenses1,DecisionTree,0.759117,0.754748,0.006054,6,9,3,best


Only 'max_depth' and 'min_samples_leaf' are considered.

In [46]:
# combinations of model to check (algorithm + parameters)
dataproc = {'log10Expenses': log10Expenses, 'log10Expenses1': log10Expenses1} 
experiments = {
    'model': ['DecisionTree'],
    'algorithms': [DecisionTreeClassifier(criterion='gini')],
    'parameters': [{'max_depth': np.arange(5, 8),
                    'min_samples_leaf': np.arange(5,15),
                    'splitter': ['best','random']}]
}
results = run_experiments(dataproc, experiments, X_train, y_train)

Results are worst than previous ones so three hyperparameters should be considered for the optimization.

In [47]:
df = pd.DataFrame(results).drop(['features'],axis=1)
df_max3 = df.sort_values(['feature_name', 'test_score'], ascending=[True, False]).groupby('feature_name').head(3)
expand_parameters(df_max3)

,feature_name,model,train_score,test_score,fit_time,max_depth,min_samples_leaf,splitter
20,log10Expenses,DecisionTree,0.796216,0.776260,0.015406,6,5,best
22,log10Expenses,DecisionTree,0.795957,0.776260,0.015676,6,6,best
42,log10Expenses,DecisionTree,0.801335,0.776145,0.017971,7,6,best
88,log10Expenses1,DecisionTree,0.759117,0.754748,0.006334,6,9,best
89,log10Expenses1,DecisionTree,0.759117,0.754748,0.006303,6,9,random
86,log10Expenses1,DecisionTree,0.759232,0.754633,0.006203,6,8,best


New set of parameters is tested. 'CrySleep' and 'Age12' or 'Age' is also considered with log10Expenses dataset.

In [54]:
log10ExpensesCryoSleep = ['CryoSleep_True'] + log10_expenses_features + ['Deck_B','Side_S'] + main_homeplanet + main_destination
log10ExpensesCryoSleepAge12 = ['CryoSleep_True','Age12'] + log10_expenses_features + ['Deck_B','Side_S'] + main_homeplanet + main_destination
log10ExpensesCryoSleepAge = ['CryoSleep_True','Age'] + log10_expenses_features + ['Deck_B','Side_S'] + main_homeplanet + main_destination

'max_depth', 'min_samples_leaf' and 'min_samples_split' are considered. 'splitter' is set to 'random'

In [57]:
# combinations of model to check (algorithm + parameters)
dataproc = {'log10ExpensesCryoSleep': log10ExpensesCryoSleep, 
            'log10ExpensesCryoSleepAge12': log10ExpensesCryoSleepAge12, 
            'log10ExpensesCryoSleepAge': log10ExpensesCryoSleepAge} 
experiments = {
    'model': ['DecisionTree'],
    'algorithms': [DecisionTreeClassifier(criterion='gini', splitter='random')],
    'parameters': [{'max_depth': np.arange(6, 20),
                    'min_samples_leaf': np.arange(5,20),
                    'min_samples_split': np.arange(5,15)}]
}
results = run_experiments(dataproc, experiments, X_train, y_train)

Better scores are achieved when 'CryoSleep' and 'Age12' is added. 'Age12' has more information than 'Age'. Best model use 'log10ExpensesCryoSleepAge12' dataset and has these hyperparameters:

- max_depth = 17
- min_samples_leaf = 16
- min_samples_split = 10

test_score = 0.787

In [58]:
df = pd.DataFrame(results).drop(['features'],axis=1)
df_max3 = df.sort_values(['feature_name', 'test_score'], ascending=[True, False]).groupby('feature_name').head(3)
expand_parameters(df_max3)

,feature_name,model,train_score,test_score,fit_time,max_depth,min_samples_leaf,min_samples_split
892,log10ExpensesCryoSleep,DecisionTree,0.791787,0.785116,0.007845,11,19,7
844,log10ExpensesCryoSleep,DecisionTree,0.791844,0.782816,0.008553,11,14,9
1179,log10ExpensesCryoSleep,DecisionTree,0.791816,0.782701,0.008365,13,17,14
4912,log10ExpensesCryoSleepAge,DecisionTree,0.794749,0.783965,0.008448,10,16,7
5519,log10ExpensesCryoSleepAge,DecisionTree,0.798689,0.783506,0.009130,14,16,14
6269,log10ExpensesCryoSleepAge,DecisionTree,0.798171,0.782931,0.008751,19,16,14
3895,log10ExpensesCryoSleepAge12,DecisionTree,0.795985,0.786729,0.008811,17,19,10
3865,log10ExpensesCryoSleepAge12,DecisionTree,0.798085,0.786728,0.008613,17,16,10
4038,log10ExpensesCryoSleepAge12,DecisionTree,0.795784,0.786613,0.008365,18,18,13


In [ ]:
# TODO list of list of parameters --> All combinations from list of list minimum with two parameters

In [70]:
from itertools import product

def constrained_combinations(list_of_lists):
    # Generate all possible combinations (including None for "no element")
    expanded = [lst + [None] for lst in list_of_lists]
    
    # Compute Cartesian product (all possible combinations)
    all_combinations = product(*expanded)
    
    # Filter combinations to:
    # 1. Exclude None-only selections
    # 2. Remove None placeholders
    # 3. Ensure at least 2 elements
    result = [
        [item for item in combo if item is not None]
        for combo in all_combinations
        if sum(1 for item in combo if item is not None) >= 2
    ]
    
    return result

In [ ]:
# TODO How it works?

In [82]:
list_of_lists = [['T','R'],['A',['D','K']],['F','E']]

expanded = [lst + [None] for lst in list_of_lists]

print(expanded)
    # Compute Cartesian product (all possible combinations)
all_combinations = product(*expanded)
print(all_combinations)
    
#    # Filter combinations to:
#    # 1. Exclude None-only selections
#    # 2. Remove None placeholders
#    # 3. Ensure at least 2 elements
result = [
        [item for item in combo if item is not None]
        for combo in all_combinations
        if sum(1 for item in combo if item is not None) >= 2
    ]
print(result)    
#    return result

sum(list_of_lists, []) # flatten a list of lists

[['T', 'R', None], ['A', ['D', 'K'], None], ['F', 'E', None]]
[['T', 'A', 'F'], ['T', 'A', 'E'], ['T', 'A'], ['T', ['D', 'K'], 'F'], ['T', ['D', 'K'], 'E'], ['T', ['D', 'K']], ['T', 'F'], ['T', 'E'], ['R', 'A', 'F'], ['R', 'A', 'E'], ['R', 'A'], ['R', ['D', 'K'], 'F'], ['R', ['D', 'K'], 'E'], ['R', ['D', 'K']], ['R', 'F'], ['R', 'E'], ['A', 'F'], ['A', 'E'], [['D', 'K'], 'F'], [['D', 'K'], 'E']]


In [71]:
# TODO Could this be improved?
onehot_homeplanet = (list(filter(lambda x: any(map(lambda y: y in x,['HomePlanet_'])), df.columns)))
main_homeplanet = list(filter(lambda x: any(map(lambda y: y not in x, ['Mars'])),onehot_homeplanet))
onehot_destination = (list(filter(lambda x: any(map(lambda y: y in x,['Destination_'])), df.columns)))
main_destination = list(filter(lambda x: any(map(lambda y: y not in x, ['PSO'])),onehot_destination))
onehot_deck = (list(filter(lambda x: any(map(lambda y: y in x,['Deck_'])), df.columns)))
expenses_features = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
log10_expenses_features = ['log10_RoomService', 'log10_FoodCourt', 'log10_ShoppingMall', 'log10_Spa', 'log10_VRDeck']
log10_expenses_features1 = ['log10_RoomService1', 'log10_FoodCourt1', 'log10_ShoppingMall1', 'log10_Spa1', 'log10_VRDeck1']

list_of_features = [['Age','Age12'], 
                    ['CryoSleep'], 
                    ['Side_S'], 
                    [expenses_features, log10_expenses_features, 'log10_TotalExpenses', log10_expenses_features1, 'log10_TotalExpenses1'],
                    ['Deck_B',['Deck_B','Deck_C','Deck_E','Deck_F'], onehot_deck],
                    [main_homeplanet],
                    [main_destination]
                   ]

In [75]:
lst = constrained_combinations(list_of_features)
# TODO Flatten lists
# TODO Convert into dictionary
len(lst)

1137

In [11]:
# TODO Implement best parameter
# TODO Use more Deck_X
# TODO Use less HomePlanet
# TODO Use less Destination
# TODO Video Imputation Kaggle
# TODO Random Forest
# TODO Feature importance
# TODO Boosted Tree sklearn
# TODO XGBoost
# TODO HistGradienteBoostingClassifier

In [57]:
best_model = pd.DataFrame(results).sort_values('acc_val', ascending=False).iloc[0,:].to_dict()
for key, value in best_model.items():
    print(f'{key}: {value}')

features: ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Destination_55 Cancri e', 'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars', 'CryoSleep_True']
algorithm: DecisionTreeClassifier(splitter='random')
parameters: {'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 4}
acc_val: 0.7839655609383592
